# Content-based (на основе описания групп)

In [1]:
import vk                                                                       
import time                                                                     
import pickle                                                                   
import numpy as np 
import collections
import itertools
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from collections import defaultdict                                             
                                                                                
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)

In [ ]:
all_members = {}
all_members.update(load_obj('./Anya_persons'))
all_members.update(load_obj('./Dasha_persons'))
all_members.update(load_obj('./Maks_persons'))
print(len(all_members))

In [3]:
users_with_no_inf = []
groups_l = set()
for pers in all_members:
    try:
        groups_l.update(all_members[pers]['groups']) 
    except Exception: users_with_no_inf.append(pers)
groups_l = list(groups_l)

In [4]:
for pers in users_with_no_inf:
    del all_members[pers]

In [7]:
print(len(groups_l))

1141947


In [7]:
groups = {idx:group for idx, group in enumerate(groups_l)}
reverse_groups = {group:idx for idx, group in enumerate(groups_l)}

In [8]:
print(max(groups.keys()))
print(max(reverse_groups.keys()))

1141946
189802470


In [9]:
users_l = list(all_members.keys())
users = {idx:user for idx, user in enumerate(users_l)}
reverse_users = {user:idx for idx, user in enumerate(users_l)}

In [10]:
print(len(all_members))
print(len(groups))

46181
1141947


In [ ]:
import vk
import time
members_set = set()
token = 'f3c81a9e09ad036126c26fbd2663e9868d431160b695c5cf676ad595258dd9a3a64c36fd85a0f2912acb5'
session = vk.Session(access_token = token)
api = vk.API(session) 
descr_of_groups = []
all_inf_about_groups = []
for i in range(2283):
    time.sleep(1)
    all_inf_about_groups.extend(api.groups.getById(group_ids = groups_l[i*500:i*500+500], fields = 'description', v = 5.103))

In [18]:
all_inf_about_groups.extend(api.groups.getById(group_ids = groups_l[1141500:-1], fields = 'description', v = 5.103))

In [30]:
dict_tmp = dict()
list_title = []
tmp = 0
for j in all_inf_about_groups:
    try:
        if j['description'] == '': dict_tmp[j['id']] = j['name']
        else : dict_tmp[j['id']] = j['description']
    except Exception: dict_tmp[j['id']] = j['name']

In [37]:
dict_tmp[188743678] = api.groups.getById(group_ids = 188743678, fields = 'description', v = 5.103)[0]['description']

In [39]:
print(len(dict_tmp ))

1141947


In [41]:
list_title = []
for i, val in enumerate(groups_l):
    list_title.append(dict_tmp[val])

In [49]:
import gensim
import re
list_title = [gensim.utils.simple_preprocess(re.sub("[^a-zA-Z]+", " ", i.lower())) 
                           for i in list_title]

In [50]:
# build vocabulary and train model
model = gensim.models.Word2Vec(
        list_title,
        size=200,
        window=10,
        min_count=1,
        workers=10,
        iter=100)

In [51]:
def avg_feature_vector(words, model, num_features, index2word_set):
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [52]:
index2word_set = set(model.wv.index2word)

In [53]:
item_feature_embeddings = np.zeros((len(list_title), 200))
for i in range(item_feature_embeddings.shape[0]):
    item_feature_embeddings[i] = avg_feature_vector(list_title[i], model, 200, index2word_set)

In [54]:
def precision_for_one(recomend, user_l):
    recomend = set(recomend)
    groups_of_user = set(all_members[user_l]['groups'])
    TP = len(recomend&groups_of_user)
    return TP/6.

In [72]:
import random
users_for_test = random.sample(users_l, 100)

In [102]:
#косинусное расстояние между эмбедингами описания групп
def content_based_recomendation(user_l, item_feature_embeddings):
    groups_for_recommend = []
    groups_of_user = all_members[user_l]['groups']
    groups_of_user_revers = []
    for i in groups_of_user[0:6]:
         groups_of_user_revers.append(reverse_groups[i])
    for i in groups_of_user_revers:
        metrics = list(cosine_similarity(item_feature_embeddings[i].reshape(1, -1),  item_feature_embeddings)[0])   
        sorted_keys = sorted(range(len(metrics)), key=metrics.__getitem__, reverse = True)
        for j, val in enumerate(sorted_keys):
            if val != i and (groups[val] not in groups_for_recommend)  and groups[val]!=2 and (groups[val] not in groups_of_user):  
                best_group = val
                break
        #if i != sorted_keys[0]: best_group = sorted_keys[0]
        #else: best_group = sorted_keys[1]
        groups_for_recommend.append(groups[best_group])
        #groups_for_recommend.append(best_group)
    return groups_for_recommend


In [ ]:
list_of_prec = []
for user_l in users_for_test:
    recomend = content_based_recomendation(user_l, item_feature_embeddings)    
    list_of_prec.append(precision_for_one(recomend, user_l))

In [101]:
sum(list_of_prec)/len(list_of_prec)

0.003333333333333333

In [104]:
# тест для себя
content_based_recomendation(9183085, item_feature_embeddings)   

[18874368, 81671907, 44610648, 1, 3, 97915820]

In [84]:
friends_for_test = []

In [85]:
with open('All_friends.txt' , 'r') as f:
    for line in f:
        friends_for_test.append(int(line))     

In [89]:
len( friends_for_test)

46

In [92]:
dict_friends_test = dict()
for pers in friends_for_test:
    if pers in users_l:
        dict_friends_test[pers] = dict()
        dict_friends_test[pers]['groups'] = content_based_recomendation(pers, item_feature_embeddings)   
        dict_friends_test[pers]['method'] = 4

In [94]:
with open('Dasha_recs_4.pkl', 'wb') as f:
    pickle.dump(dict_friends_test, f)